In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.interpolate import splprep, splev
%matplotlib inline

import triangle
import triangle.plot

In [4]:
# Load the image and convert to B&W
I = plt.imread('hawk.png')
I = I.sum(axis=2)
I = np.where(I!=0, 1, 0)
I = np.uint8(I)
np.where?

## load image, find boundary

In [5]:
# Load the image and convert to B&W
I = plt.imread('hawk.png')
I = I.sum(axis=2)
I = np.where(I!=0, 1, 0)
I = np.uint8(I)

xmax = I.shape[1]
ymax = I.shape[0]

# blur, exand, then contract the image to smooth the lines
blur=((3,3),1)
erode=(3, 3)
dilate=(5, 5)

I = cv2.GaussianBlur(I , blur[0], blur[1])
I = cv2.erode(I, np.ones(erode))
I = cv2.dilate(I, np.ones(dilate))
I *= 255

# find the contours
I, contours, _ = cv2.findContours(I, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

I2 = 0 * I.copy()
contour = contours[0]
for contour in contours:
    for i, j in contour[:,0]:
        I2[j,i] = 127

# get the image as coordinates
tmpy, tmpx = np.where(I>0)
IXY = np.vstack((tmpx, tmpy)).T
IXY = np.array([0,1]) - IXY / np.array([-xmax, ymax])

#only 4 matters (the longest)
#rescale XY
XY = np.array([0,1]) - contours[4][:,0] / np.array([-xmax, ymax])
#XY = XY[:-1,:]

# make a spline
n = 500
tck, u = splprep(XY.T, u=None, s=0.0)
u_new = np.linspace(u.min(), u.max(), n)
x_new, y_new = splev(u_new, tck, der=0)

XYnew = np.vstack((x_new, y_new)).T
nlist = np.arange(0,XYnew.shape[0])
segments = np.vstack((nlist, np.roll(nlist, -1))).T

#plt.scatter(IXY[:,0], IXY[:,1])
plt.plot(XY[:,0], XY[:,1], 'r-', lw=4)
plt.plot(x_new, y_new, 'b--s')

ValueError: not enough values to unpack (expected 3, got 2)

## Make triangulation

In [ ]:
t = triangle.triangulate({'vertices': XYnew, 'segments': segments}, 'pq30a.01D')

In [ ]:
triangle.plot.plot(plt.axes(), **t)

## Try PyAMG

In [ ]:
import scipy.sparse as sparse
triangles = t['triangles']

row = triangles.ravel()
col = np.roll(triangles, -1, axis=1).ravel()
data = 1.0*np.ones((len(row),))

L = sparse.coo_matrix((data, (row, col)))
# symmetrize
L = L.T + L
# set to all minus ones
L.data[:] = -1
# zero diagonal
L.setdiag(0.0)
# collect totals
diag = -np.array(L.sum(axis=1).ravel()).ravel()
# set to minus totals
L.setdiag(diag)
# find Dinv
n = L.shape[0]
Dinv = sparse.spdiags(1.0/diag, [0], n, n, format='csr')
L = Dinv.dot(L.dot(Dinv))
L = L.tocsr()

In [ ]:
import pyamg
ml = pyamg.smoothed_aggregation_solver(L, max_levels=2, keep=True, max_coarse=10)

AggOp = ml.levels[0].AggOp.tocsc()
edges = []
for c in range(AggOp.shape[1]):
    row = AggOp.getcol(c).indices
    # add to edges any edge in the aggregate
    for i in row:
        for j in row:
            if i != j and L[i,j]:
                edges.append([i,j])

#AggOp = AggOp.T                
AggOp = AggOp.tocsr()
col = AggOp.indices
aggtris = [t for t in triangles if col[t[0]]==col[t[1]] and col[t[1]]==col[t[2]]]

In [ ]:
print(triangles.shape)
print(len(aggtris))

## Make logo

In [ ]:
x = t['vertices'][:,0]
y = t['vertices'][:,1]
triangles = t['triangles']

zfaces = np.array([np.linalg.norm([x[t[0]], y[t[0]]]) for t in triangles])
plt.figure(dpi=200)
plt.tripcolor(x, y, triangles, facecolors=zfaces, edgecolors='b', cmap=plt.cm.binary)

# plot aggregate edges
#for e in edges:
#    plt.plot([x[e[0]], x[e[1]]], [y[e[0]], y[e[1]]], 'r-', lw=1.0)
   
# plot aggregate triangles
#plt.plot(x[triangles[1]], y[triangles[1]], 'bo')
plt.tripcolor(x, y, aggtris, facecolors=zfaces.max()*np.ones((len(aggtris,))), cmap=plt.cm.Blues_r)

# plot title
plt.text(0.4, -0.1,
         'RAPtor: parallel algebraic multigrid',
         fontsize=28,
         family='sans-serif',
         style='italic',
         color=[0.3, 0.3, 0.3],
        fontweight='bold')
plt.axis('equal')
plt.axis('off')
plt.tight_layout()

plt.savefig('raptor-logo.png',
            dpi=200,
            bbox_inches='tight',
            pad_inches=0,
            transparent=True)